In [1]:
#libraries
import pandas as pd
import numpy as np
from IPython import display
# from pymongo_test_insert import get_database #local library
import pymongo
from bson.objectid import ObjectId  #work with Object Id in Mongo
# import get_collections #currently not working
# dbname = get_database()

## Overview of data
There are 37 relational tables like the following (download [here](
https://drive.google.com/file/d/1HomOYSpsXYQSAcCMMoW3oWu1FLN0r6pD/view?usp=sharing) for a better look) ![Data Schema](./db schema.png)

Several tables are in interests such as __*students*__, __*courses*__, __*student_enrolls*__, __*course_contents*__

In [2]:
#getting collections from db "e-learning": 
db_connect = pymongo.MongoClient('localhost', 27017) # change local host here
database_name1 = "elearning281021" # change dbname here
database_name0 = "elearning"
database1 = db_connect[database_name1] #original data = 0
database0 = db_connect[database_name0]
collections1 = database1.collection_names(include_system_collections=False)
collections0 = database0.collection_names(include_system_collections=False)
print(len(collections1), len(collections0)) # number of collections

37 37


C:\Users\Admin\AppData\Local\Temp/ipykernel_17128/1390454847.py:7: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  collections1 = database1.collection_names(include_system_collections=False)
C:\Users\Admin\AppData\Local\Temp/ipykernel_17128/1390454847.py:8: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  collections0 = database0.collection_names(include_system_collections=False)


In [3]:
# getting and converting data from each collections to dataframe
for i in collections1:
    collection_name = database1[i]
    item_details = collection_name.find()
    content = []
    for item in item_details:
        content.append(item)
    globals()["df_" + i] = pd.DataFrame(content) #generating dataframe files
    pd.DataFrame(content).to_csv("./Data-28-10-21_csv/df_"+ i + ".csv", index = False) 
    # print("df_" + i)

## Enrollment
- A student can choose to enroll in multiple courses, resulting in all number of courses, not students, being registered.
- Run a code chunck below we get:

By the way bong, I've been looking at the dashboard and the raw data that we have so far, and I found out that there are a lot more important indications that we miss out but can reproduce. For example, we can identify "active learners" by their activities in class (answer quizzes, discussion) as well as in the platform in general (numbers of class s/he has enrolled, questions bank participation). Also, we can 

In [4]:
#check number of student enrolled [student_enrolls]
print("number of total courses being enrolled: " + str(len(df_student_enrolls["students"])))
print("number of all UNIQUE courses being enrolled: " + str(len(df_student_enrolls["courses"].unique())) + " courses")
print("number of students enrolled in the courses: " + str(len(df_student_enrolls["students"].unique())) + " students")

number of total courses being enrolled: 1306
number of all UNIQUE courses being enrolled: 383 courses
number of students enrolled in the courses: 507 students


## Active Learners
### Defining __active students__ status:

We proxy the variable based on 2 main conditions as following:

➀ __watch duration__: 

- In seconds we identify those who have watched the video at least 70%-80% threshold, resulting in 1 __view count__.

➁　__student activities__: 

- how active students engage in replying discussion, anwsering quizzes, asking questions. 

- number of completed class (df_student_activities["is_completed"])

➂ __number of classes enrolled__: not really convincing at all, but here for only neccessary condition. 

➃ __ranking__: 

- features top 5 students every months (5 **active users**/month)


In [16]:
#Define video view counts in [student_activities]
df_course_contents['content_duration']

0           NaN
1           NaN
2        1790.0
3           NaN
4           NaN
          ...  
32178       NaN
32179       NaN
32180       NaN
32181       NaN
32182       NaN
Name: content_duration, Length: 32183, dtype: float64

In [104]:
# Accessing lesson with respective ID in 2 ways
#Way 1
courses_content.loc[courses_content["_id"] == ObjectId("6098a809ada1c61054ab9baf")][["view_count", "course_contents"]].iloc[:,1]
#Way 2
#find courses name in according with respective course ID
## courses_content.loc[courses_content["_id"] == "condition to identify course id"]["name"]
# courses_content.loc[courses_content["_id"] == course_contents_content.loc[course_contents_content["_id"] == ObjectId("60b85eb9b7e0ea67ef830e68")]["courses"].iloc[0]]["name"]

5    [6098c5ee572591105f050865, 6098a897e934b71019f...
Name: course_contents, dtype: object

In [100]:
#Course Enrollment: get which course are most enrolled
## get connect with courses_content (id)
student_enrolls_content["courses"].describe() # 6098a809ada1c61054ab9baf with 18 frq

#Getting name of which course gets enrolled the most
courses_content.loc[courses_content["enrolled_count"] == student_enrolls_content["courses"].describe()[3], "name"]

5    មេរៀនទី៧ ស្ថិតិ
Name: name, dtype: object

In [109]:
courses_content["course_contents"][0]

[ObjectId('6098a6f3a3d06d13536860c2'),
 ObjectId('6098a6fa98a1111330b75837'),
 ObjectId('6098a7efe934b71019f583d7'),
 ObjectId('6098ade9adadc310362e413c'),
 ObjectId('6098b266572591105f050825'),
 ObjectId('6098b37be9db40107d742732'),
 ObjectId('6098c0ad946da8130d466d58'),
 ObjectId('6098e07e01c156105d56a58e'),
 ObjectId('6098e8b02f4c41131d66b8da'),
 ObjectId('6098ebdb84286213144b6d88'),
 ObjectId('6098edfeadadc310362e531c')]

In [120]:
len(student_enrolls_content["students"].unique())

133

## Cleaning procedure

- Checking relationships between [courses] and [course_contents]
    - for each __"id"__ in __[courses]__ can have multiple __"course_contents"__
    - __"courses"__ in __[student_enrolls]__ <-> __"id"__ in __[courses]__
   

In [5]:
#Satya's ID
60981ece1e4b5a12ff6bf41d
6098d8a0a33c5c13108d8ee0

In [265]:
#number and list of registered courses for each student ID
list_courses_IDregistered = student_enrolls_content.loc[student_enrolls_content["students"] == ObjectId("6098d8a0a33c5c13108d8ee0")]["courses"].tolist()
#for one registered course (ជំពូក) id refers to multiple lessons inside (មេរៀន)
[courses_content.loc[courses_content["_id"] == i]["name"].tolist() for i in list_courses_IDregistered]

[['ជំពូកទី៥: សមីការឌីផេរ៉ង់ស្យែល'],
 ['ជំពូកទី១ តារាងខួបនៃធាតុគីមី'],
 ['ជំពូកទី១៖ តារាងខួបនៃធាតុគីមី'],
 ['មេរៀនទី៧ ស្ថិតិ'],
 ['មេរៀនទី១៨ សូលីត'],
 ['ជំពូកទីី៤ គីមីសរីរាង្គ'],
 ['មេរៀនទី១៧ ពហុកោណ'],
 ['ជំពូកទី៥ អនុគមន៍'],
 ['មេរៀនទី១៧ ពហុកោណ'],
 ['ជំពូកទី៤ អេឡិចត្រូម៉ាញេទិច'],
 ['មេរៀនទី៨ ការសម្រេចចិត្ត'],
 ['ជំពូកទី៤ ដង្ហើម និងដំណឹកនាំ'],
 ['មេរៀនទី៧ ការជឿជាក់'],
 ['ជំពូកទី៣ ទឹក និងសូលុយស្យុង'],
 ['មេរៀនទី៧ ប្រពៃណីទំនៀមទម្លាប់'],
 ['មេរៀនទី១០ សមីការនៃបន្ទាត់'],
 ['ជំពូកទី៣ អគ្គិសនី'],
 ['ជំពូកទី៥ អាល់កុល និងថ្នាំជក់'],
 ['មេរៀនទី៥ ល្ខោន'],
 ['ជំពូកទី១ ពិភពលោក'],
 ['A'],
 ['ជំពូកទី១ ប្រវត្តិសាស្រ្តពិភពលោក'],
 ['មេរៀនទី៨ ប្រូបាប'],
 ['ជំពូកទី១ រស្មីសំយោគ'],
 ['មេរៀនទី១ ចំនួនអសនិទាន'],
 ['មេរៀនទី៧ រង្វាស់រង្វាល់'],
 ['មេរៀនទី១ ចំនួនគត់'],
 ['មេរៀនទី២ វិធីគុណ'],
 ['មេរៀនទី១ សេចក្ដីស្រលាញ់'],
 ['ជំពូក១ មេកានិច'],
 ['មេរៀនទី១ អក្សរសិល្ប៍ខេមរនិយម'],
 ['ជំពូកទី៤ អាំងតេក្រាល'],
 ['មេរៀនទី១ ទំហំ'],
 ['ជំពូកទី១ ប្រវត្តិសាស្ត្រពិភពលោក'],
 ['ជំពូក១ ស្វុីត'],
 ['ជំពូកទី១ កម្ដៅ'],
 ['ជំពូកទី៤ នាទី

In [440]:
###getting course content 
### ?? why one course has multiple IDs?? --> 
content_in_course = courses_content.loc[courses_content["name"] == 'ជំពូកទី៥: សមីការឌីផេរ៉ង់ស្យែល']["course_contents"].tolist()
for i in range(len(content_in_course)):
    for j in range(len(content_in_course[i])):
        print(course_contents_content.loc[course_contents_content["_id"] == content_in_course[i][j]]["name"].tolist())
        
# course_contents_content.loc[course_contents_content["_id"] == content_in_course[0][0]]["name"].tolist()
#         course_contents_content.loc[course_contents_content["_id"] == content_in_course[i][j]]["name"].tolist()

['មេរៀនទី១: សមីការឌីផេរ៉ង់ស្យែលលីនេអ៊ែលំដាប់ទី១']
['មេរៀនសមីការឌីផេរ៉ង់ស្យែលលីនេអ៊ែលំដាប់ទី១(Part1)']
['មេរៀនសមីការឌីផេរ៉ង់ស្យែលលីនេអ៊ែលំដាប់ទី១ (Part 2-End)']
['មេរៀនទី២: សមីការឌីផេរ៉ង់ស្យែលលីនេអ៊ែលំដាប់ទី២']
['មេរៀនសមីការឌីផេរ៉ង់ស្យែលលីនេអ៊ែលំដាប់ទី២']
['លំហាត់គំរូ Part1(1&2)']
['លំហាត់គំរូ Part2(3)']


In [448]:
# len(student_activities_content.loc[student_activities_content["is_completed"] == True]["students"].unique())
import numpy as np
np.nansum(student_activities_content["watch_duration"])/3600

175.7

In [296]:
#use subject id in [course] to get subject name
subjects_content.loc[subjects_content["_id"] == ObjectId(courses_content["subjects"].unique().tolist()[4])]["name"]

3    រូបវិទ្យា
Name: name, dtype: object

In [451]:
#find students with id
students_content.loc[students_content["_id"] == ObjectId("6098dab798a1111330b7623c")][["first_name", "last_name"]]

,first_name,last_name
10,Satsya,Nhoem


In [297]:
students_content.columns

Index(['__v', '_id', 'address', 'createdAt', 'date_of_birth',
       'default_schools', 'education_levels', 'first_name', 'gender',
       'last_name', 'level_types', 'logins', 'profile_image', 'roles',
       'status', 'updatedAt'],
      dtype='object')

In [304]:
courses_content.loc[courses_content["_id"]==favorite_courses_content.loc[favorite_courses_content["students"]==ObjectId("6098d70301c156105d56a587")]["courses"].tolist()[0]]["name"]
#getting the most popular course

5    មេរៀនទី៧ ស្ថិតិ
Name: name, dtype: object

In [353]:
# Getting top 3 students who registered the most
from collections import Counter
list_top_3 = Counter(student_enrolls_content["students"].tolist()).most_common()[:3]
# sorted(range(len(student_enrolls_content["students"])), key= lambda i: student_enrolls_content["students"][i], reverse=True)[:3]
print(list_top_3)
for j in list_top_3:
    print(students_content.loc[students_content["_id"]==j[0]][["first_name","last_name"]])

[(ObjectId('6098d8a0a33c5c13108d8ee0'), 40), (ObjectId('6095728f83c32a0468b97de0'), 16), (ObjectId('60d68de5147675465092b0ae'), 15)]
  first_name last_name
9     Satsya     Nhoem
  first_name last_name
0      lalin      jame
   first_name last_name
72    ជ័យខេមា        ឯម


In [354]:
students_content.columns

Index(['__v', '_id', 'address', 'createdAt', 'date_of_birth',
       'default_schools', 'education_levels', 'first_name', 'gender',
       'last_name', 'level_types', 'logins', 'profile_image', 'roles',
       'status', 'updatedAt'],
      dtype='object')

In [343]:
#getting students who registered the most in [student_enrolls]
# students_content.loc[students_content["_id"]==student_enrolls_content["students"].describe()[2]][["first_name","last_name"]]
students_content.loc[students_content["_id"]==student_enrolls_content["students"].describe()[2]][["first_name","last_name"]]

,first_name,last_name
9,Satsya,Nhoem


In [329]:
student_enrolls_content["students"][0]

ObjectId('6095728f83c32a0468b97de0')

In [362]:
len(student_enrolls_content["students"].tolist())

394

In [361]:
len(student_enrolls_content["students"].unique())

133

## Checking if students are actually learning
- Students might register but not actually follow the lecture
- Follow student activities, specifically looking into "watch duration" if that surpasses a certain threshold, we define that as "__learner__".
- "__active learner__" those who *keep coming* and *active on discussion*, longer watch duration


In [452]:
roles_content

,_id,role_name
0,1.0,admin
1,3.0,teacher
2,5.0,reviewer
3,2.0,school
4,4.0,student


In [453]:
import pandas as pd
import numpy as np
from scipy.special import expit
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

style.use("fivethirtyeight")

np.random.seed(123)
n = 100
tuition = np.random.normal(1000, 300, n).round()
tablet = np.random.binomial(1, expit((tuition - tuition.mean()) / tuition.std())).astype(bool) # create binary variable
enem_score = np.random.normal(200 - 50 * tablet + 0.7 * tuition, 200)
enem_score = (enem_score - enem_score.min()) / enem_score.max()
enem_score *= 1000

In [454]:
tablet

array([False,  True, False, False, False, False,  True, False,  True,
       False,  True,  True,  True, False,  True, False,  True,  True,
        True, False, False,  True, False,  True, False, False,  True,
       False,  True, False, False, False, False, False, False,  True,
       False, False, False,  True, False, False,  True, False, False,
       False,  True, False,  True,  True, False,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True, False, False, False, False,  True,  True,
        True, False,  True,  True, False,  True, False, False,  True,
       False,  True, False, False, False, False, False,  True, False,
       False, False,  True,  True,  True,  True,  True, False,  True,
        True])

In [455]:
(data
 .assign(class_format = np.select(
     [data["format_ol"].astype(bool), data["format_blended"].astype(bool)],
     ["online", "blended"],
     default="face_to_face"
 ))
 .groupby(["class_format"])
 .mean())

Object `np.random.binomal` not found.
